In [ ]:
import dlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
bulucu= dlib.get_frontal_face_detector()
bicim= dlib.shape_predictor('C:/Users/Burcu/Desktop/modeller/shape_predictor_5_face_landmarks.dat') 

In [ ]:
foto = dlib.load_rgb_image('C:/Users/Burcu/Desktop/fotolar/07.jpg')

plt.figure(figsize=(26, 20))
plt.imshow(foto)

In [ ]:
fotosonuc = foto.copy()

blc= bulucu(foto, 1)

if len(blc) == 0:
    print('Yüz Bulunamadı!')

fig, ax = plt.subplots(1, figsize=(16, 10))

for det in blc:
    k, l, m, n = det.left(), det.top(), det.width(), det.height()

    rect = patches.Rectangle((k, l), m, n, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

ax.imshow(fotosonuc)

In [ ]:
fig, ax = plt.subplots(1, figsize=(16, 10))
nesne = dlib.full_object_detections()
for detection in blc:
    s = bicim(foto, detection)
    nesne.append(s)
    for point in s.parts():
        circle = patches.Circle((point.x, point.y), radius=3, edgecolor='r', facecolor='r')
        ax.add_patch(circle)
ax.imshow(fotosonuc)

In [ ]:
yuzler = dlib.get_face_chips(foto, nesne, size=256, padding=0.18)

fig, axes = plt.subplots(1, len(yuzler)+1, figsize=(20, 16))

axes[0].imshow(foto)

for p, yuz in enumerate(yuzler):
    axes[p+1].imshow(yuz)

In [ ]:
def hizali_yuzler(foto):
    blc = bulucu(foto, 1)
    
    nesne = dlib.full_object_detections()

    for detection in blc:
        s = bicim(foto, detection)
        nesne.append(s)
    yuzler = dlib.get_face_chips(foto, nesne, size=256, padding=0.35)
    return yuzler

# çoklu yüzde hizalama işlemi kontrolü
kontrol_foto = dlib.load_rgb_image('C:/Users/Burcu/Desktop/fotolar/07.jpg')
kontrol_yuzler = hizali_yuzler(kontrol_foto)
fig, axes = plt.subplots(1, len(kontrol_yuzler)+1, figsize=(20, 16))
axes[0].imshow(kontrol_foto)
for i, yuz in enumerate(kontrol_yuzler):
    axes[i+1].imshow(yuz)

In [ ]:
tf.disable_v2_behavior()
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

saver = tf.train.import_meta_graph('C:/Users/Burcu/Desktop/modeller/model.meta')
saver.restore(sess, tf.train.latest_checkpoint('C:/Users/Burcu/Desktop/modeller'))
graph = tf.get_default_graph()

X = graph.get_tensor_by_name('X:0') #KAYNAK FOTOĞRAF
Y = graph.get_tensor_by_name('Y:0') #AKTARILAN FOTOĞRAF 
Xs = graph.get_tensor_by_name('generator/xs:0') #SONUC

In [ ]:
def islemoncesi(foto):
    return foto.astype(np.float32) / 127.5 - 1.

def islemsonrasi(foto):
    return ((foto + 1.) * 127.5).astype(np.uint8)

In [ ]:
foto1 = dlib.load_rgb_image('C:/Users/Burcu/Desktop/fotolar/07.jpg')
foto1_yuzler = hizali_yuzler(foto1)

foto2 = dlib.load_rgb_image('C:/Users/Burcu/Desktop/fotolar/makyaj/makeup03.jpg')
foto2_yuzler = hizali_yuzler(foto2)

fig, axes = plt.subplots(1, 2, figsize=(16, 10))
axes[0].imshow(foto1_yuzler[0])
axes[1].imshow(foto2_yuzler[0])

In [ ]:
src_foto = foto1_yuzler[0]
ref_foto = foto2_yuzler[0]

X_foto = islemoncesi(src_foto)
X_foto = np.expand_dims(X_foto, axis=0)

Y_foto = islemoncesi(ref_foto)
Y_foto = np.expand_dims(Y_foto, axis=0)

cıkıs= sess.run(Xs, feed_dict={
    X: X_foto,
    Y: Y_foto
})

cıkıs_foto = islemsonrasi(cıkıs[0])

fig, axes = plt.subplots(1, 3, figsize=(20, 10))
axes[0].set_title('KAYNAK FOTOĞRAF')
axes[0].imshow(src_foto)
axes[1].set_title('AKTARILAN MAKYAJ')
axes[1].imshow(ref_foto)
axes[2].set_title('SONUC')
axes[2].imshow(cıkıs_foto)

In [ ]:
data= np.load('ORL_faces.npz')

In [ ]:
type(data)

In [ ]:
# load the "Train Images"
x_train = data['trainX']
#normalize every image
x_train = np.array(x_train,dtype='float32')/255

x_test = data['testX']
x_test = np.array(x_test,dtype='float32')/255

# load the Label of Images
y_train= data['trainY']
y_test= data['testY']

# show the train and test Data format
print('x_train : {}'.format(x_train[:]))
print('Y-train shape: {}'.format(y_train))
print('x_test shape: {}'.format(x_test.shape))

In [ ]:
x_train, x_valid, y_train, y_valid= train_test_split(
    x_train, y_train, test_size=.05, random_state=1234,)

In [ ]:
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)

#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)

print('x_train shape: {}'.format(y_train.shape[0]))
print('x_test shape: {}'.format(y_test.shape))

In [ ]:
cnn_model= Sequential([
    Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
     Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(20, activation='softmax')  
])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

In [ ]:
cnn_model.summary()

In [ ]:
history=cnn_model.fit(
    np.array(x_train), np.array(y_train), batch_size=512,
    epochs=250,verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)),
)

In [ ]:
scor = cnn_model.evaluate( np.array(x_test),  np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()